<a href="https://colab.research.google.com/github/Stefano-t/bioinf-lab/blob/main/k562_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crr_labels epigenomic_dataset ucsc_genomes_downloader cache_decorator &> /dev/null && echo "done"

done


In [2]:
from cache_decorator import Cache
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from matplotlib.axes import Axes
from matplotlib.figure import Figure
from matplotlib.colors import ListedColormap, LogNorm
from multiprocessing import cpu_count, Pool
import matplotlib.pyplot as plt
from epigenomic_dataset import load_epigenomes

In [3]:
cell_line = "K562"
genome_assembly = "hg38"
window_size = 256

promoters_epigenomes, promoters_labels = load_epigenomes(
    cell_line = cell_line,
    dataset = "fantom",
    region = "promoters",
    window_size = window_size,
    root = "datasets"
)

enhancers_epigenomes, enhancers_labels = load_epigenomes(
    cell_line = cell_line,
    dataset = "fantom",
    region = "enhancers",
    window_size = window_size,
    root = "datasets"
)

In [32]:
def describe_dataset(
    X: pd.DataFrame,
    y: pd.DataFrame
):
    """Performe a quick report for some relevant information in the dataset.

    Parameters
    ---------- 
    X: pd.DataFrame
        The dataframe to describe.
    y: pd.DataFrame
        The labels to describe.
    """
    print(X.describe())
    print(X[:5])
    print(y.describe())
    print(y[:5])
    print("="*30, "Feature Sample ration", "="*30)
    print(f"Features/Samples ratio is: {X.shape[0] / X.shape[1]}")
    print("="*30, "NaN count", "="*30)
    print(f"Total NaN values: {X.isna().values.sum()}/{X.values.size}")
    print(f"Max NaN in a row: {X.isna().sum(axis=1).max()}/{X.shape[1]}")
    print(f"Max NaN in a feature: {X.isna().sum().max()}/{X.shape[0]}")   

In [33]:
describe_dataset(promoters_epigenomes, promoters_labels)

chrom         SMAD5         NCOA2  ...         NCOA1          KLF1
count  99881.000000  99881.000000  ...  99881.000000  99881.000000
mean       2.238571      0.657735  ...      0.699335      1.298927
std        2.650387      0.930009  ...      0.462811      1.111308
min        0.000000      0.000000  ...      0.000000      0.000000
25%        0.620000      0.410000  ...      0.370000      0.660000
50%        1.100000      0.600000  ...      0.650000      1.040000
75%        3.070000      0.800000  ...      0.970000      1.620000
max       55.770000     38.470000  ...     11.170000     46.240000

[8 rows x 429 columns]
chrom                             SMAD5  NCOA2  ZNF23  ...  NCOR1  NCOA1  KLF1
chrom chromStart chromEnd strand                       ...                    
chr1  628964     629220   +        0.14   1.00   0.19  ...   1.40   1.74  1.59
      629013     629269   +        0.14   0.97   0.20  ...   1.42   1.63  1.21
      629642     629898   +        0.00   0.11   0.11  ..

In [34]:
describe_dataset(enhancers_epigenomes, enhancers_labels)

chrom         SMAD5         NCOA2  ...         NCOA1          KLF1
count  63285.000000  63285.000000  ...  63285.000000  63285.000000
mean       0.854204      0.743733  ...      0.808300      0.942412
std        0.878816      0.334574  ...      0.520243      0.563087
min        0.000000      0.000000  ...      0.000000      0.000000
25%        0.460000      0.520000  ...      0.480000      0.610000
50%        0.720000      0.720000  ...      0.770000      0.870000
75%        1.030000      0.930000  ...      1.080000      1.170000
max       35.750000      6.470000  ...     37.360000     34.320000

[8 rows x 429 columns]
chrom                              SMAD5  NCOA2  ZNF23  ...  NCOR1  NCOA1  KLF1
chrom chromStart chromEnd  strand                       ...                    
chr10 100006381  100006637 .        0.90   0.90   1.37  ...   2.38   0.82  0.69
      100008146  100008402 .        0.73   1.06   0.74  ...   3.11   0.49  1.05
      100014418  100014674 .        0.77   0.33   0.5

In [ ]:
def drop_constant_features(X: pd.DataFrame) -> (pd.DataFrame, bool):
    """Return a DataFrame without constant features in in.

    Parameters
    ----------------
    X: pd.DataFrame
        The dataframe to process.

    Returns
    ----------------
    A tuple with the new DataFrame and a boolean value to point out if the
    new DataFrame is distinct from the input one.
    """
    new_X = X.loc[:, (df != df.iloc[0]).any()]
    return (new_X, new_X != X)

In [35]:
def impute_with_median(X: pd.DataFrame) -> pd.DataFrame:
    return df.fillna(df.median())

In [36]:
from sklearn.preprocessing import RobustScaler

def robust_scaler(X: pd.DataFrame) -> pd.DataFrame:
    return pd.DataFrame(
        RobustScaler().fit_transform(X.values),
        columns=X.columns,
        index=X.index
    )